In [ ]:
# Answer a:

import pandas as pd;
xl=pd.ExcelFile("C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\OutPuts\\m_data_1.xls")
df=xl.parse("Sheet1")
tmp=df[df['class'].isin(['ckd', 'notckd'])]
data=tmp[['hemo', 'pcv', 'sg', 'rbcc','al','class']]
data

In [ ]:
#Answer b 

import numpy as np;
byclass=data.groupby("class")
byclass.mean()


In [ ]:
intra_dist_cal=lambda x: abs(x - x.mean())
within_dist=byclass.transform(intra_dist_cal)
within_dist

In [ ]:
# Answer c:


protein=['hemo', 'pcv', 'rbcc', 'al']
cls=['ckd', 'notckd']
dist=dict()
for c in cls:
    pro=list()
    for p in protein:
        func=lambda x: abs(x-byclass.mean().ix[c,p])
        pro.append(data[p].apply(func))
    dist[c]=(pd.DataFrame(pro).transpose())
ckd=dist['ckd']
notckd=dist['notckd']
ckd.head()
notckd.head()

In [ ]:
get_ipython().magic('matplotlib inline')
import seaborn as sns
import pylab
from sklearn import *
from scipy.stats import gaussian_kde
import math
from itertools import combinations
data=pd.read_excel("C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\OutPuts\\m_data_1.xls",index_col=0)
factor=data['class']
classes=list(set(factor))
palette=['#e41a1c', '#377eb8']
color_map = dict(zip(classes,palette))
colors = factor.apply(lambda group: color_map[group])

def distance_consistency(cs_x,cs_y,avgX,avgY,outlierCS,index):
    for i in range(len(cs_x)):
        distanceCS = math.hypot(cs_x[i] - avgX[index], cs_y[i] - avgY[index])
        for counter in range(1, len(avgX)):
            distance_other_CS = math.hypot(cs_x[i] - avgX[counter], cs_y[i] - avgY[counter])                    
            if distance_other_CS < distanceCS: 
                outlierCS += 1
                break
            else: continue
    num= float(len(cs_x)-outlierCS)
    denom = float(len(cs_x))
    return((num/denom*(100)))
for rc in range(len(data.columns[0:5])):
    for group in classes:
        y = data[data['class'] == group].icol(rc).values
        ind = np.linspace(y.min(), y.max(),400)

    for r in range(len(data.columns[0:5])):
        if (rc != r):
            avgX=[]
            avgY=[]
            for group in classes:
                y = data[data['class'] == group].icol(rc).values
                x = data[data['class'] == group].icol(r).values
               
                if group == 'ckd':
                    avgX.append(np.average(x))
                    avgY.append(np.average(y))
                    cs_x=x
                    cs_y=y
                
               
                if group == 'notckd':
                    avgX.append(np.average(x))
                    avgY.append(np.average(y))
                    ts_x=x
                    ts_y=y
            print(avgX, avgY)
            
            outlierCKD = 0
            outlierNOTCKD = 0
            if (rc,r) in [(comb) for comb in combinations(range(5), 2)]:
                print('Distance consistency for:' +str(data.columns[rc])+" vs "+ str(data.columns[r]))
                ckd=distance_consistency(cs_x,cs_y,avgX,avgY,outlierCKD,0)
                notckd=distance_consistency(ts_x,ts_y,avgX,avgY,outlierNOTCKD,1)
                
                tot=(ckd+notckd)/2
                
                print(str(tot) +'%')


In [ ]:
# Question d:

get_ipython().magic('matplotlib inline')
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(style="white")
sns.set_palette("husl")
sns.set_style({'legend.frameon': True})
g = sns.PairGrid(data,hue="class",hue_kws={"marker": ["D","^"]})
# g.map_diag(sns.kdeplot)
g.map_offdiag(plt.scatter);
g.add_legend()
g.fig.subplots_adjust(top=0.9)
g.fig.suptitle('Identify CKD or NOTCKD',size=16)
g.fig.savefig('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\OutPuts\\scatter matrix for 1d.png')

